In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensor.operation.kruskal import kruskal
from tensor.operation.khatri_rao import khatri_rao
from tensor.operation.matricize import matricize
from tensor.operation.sampled_khatri_rao_prod import sampled_khatri_rao

Algorithm: CP RAND

$\text{Input:}$ $\mathcal{X} \in \mathbb{R}^{I_1 \times \cdots \times I_N}$, $\mathbf{R}$, $\mathbf{S}$

$\text{Output:}$ $\boldsymbol{\lambda}$, $\{\mathbf{A}^{(n)}\}$

$
\textbf{CPRAND} (\mathcal{X}, \mathbf{R}, \mathbf{S}):\\
\quad \text{Initialize factor matrices } \mathbf{A}^{(2)}, \dots, \mathbf{A}^{(N)}\\
\quad \textbf{repeat}:\\
\quad \quad \text{for } n = 1, \dots, N \text{ do}:\\
\quad \quad \quad \text{Define sampling operator} \mathbf{S} \in \mathbb{R}^{S \times \prod_{m \neq n} I_m}\\
\quad \quad \quad \mathbf{Z}_S \gets \text{SKR}(\mathbf{S}, \mathbf{A}^{(1)}, \dots, \mathbf{A}^{(n-1)}, \mathbf{A}^{(n+1)}, \dots, \mathbf{A}^{(N)})\\
\quad \quad \quad \mathbf{X}^T_S \gets \mathbf{S} \mathbf{X}^T_{(n)}\\
\quad \quad \quad \mathbf{A}^{(n)} \gets \underset{\mathbf{A}}{\arg \min} \lVert \mathbf{Z}_S \mathbf{A}^T - \mathbf{X}^T_S \rVert_F\\
\quad \quad \quad \text{Normalize columns of } \mathbf{A}^{(n)} \text{ and update } \boldsymbol{\lambda}\\
\quad \quad \textbf{end for}\\
\quad \textbf{until convergence}\\
\quad \text{Return } \boldsymbol{\lambda}, \{\mathbf{A}^{(n)}\}\\
$


In [2]:
def cprand(X, rank, max_iter=10, tol=1e-6):
    N = X.shape[0]
    factors = [np.random.rand(X.shape[i], rank) for i in range(X.ndim)]
    
    for iter_num in range(max_iter):
        for n in range(N):
            S = np.random.choice(N, size=rank, replace=False)
            Z_S = sampled_khatri_rao(S, factors)
            X_S = X[S]
            Z_S_inv = np.linalg.pinv(Z_S)
            factors[n] = np.dot(X_S, Z_S_inv)
        if iter_num % 10 == 0:
            print(f'iter {iter_num}: {np.linalg.norm(X - khatri_rao(*factors))}')
        if np.linalg.norm(X - khatri_rao(*factors)) < tol:
            break
    return factors


In [3]:
X = np.random.rand(100, 100, 100)
rank = 10
factors = cprand(X, rank)
print(factors[0].shape, factors[1].shape, factors[2].shape)
print(np.linalg.norm(X - khatri_rao.khatri_rao(*factors)))


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed